In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import os
import glob

In [2]:
test_features = pd.DataFrame()

f_list = []
path = './test_files'
for filename in glob.glob(os.path.join(path, '*.csv')):
    df = pd.read_csv(filename)
    dphi = np.diff(np.asarray(df['lat'].values, dtype = np.float64))
    dlambda = np.diff(np.asarray(df['lon'].values, dtype = np.float64))
    d2alt = np.diff(np.diff(np.asarray(df['alt'].values, dtype = np.float64)))
    dt = np.diff(np.asarray(df['t'].values, dtype = np.float64))
    dphi /= dt
    dlambda /= dt
    d2alt /= dt[:-1]
         
    dalt_mean = np.mean(d2alt)
    
    threshold_for_maneuver = 30
    is_maneuver = 0
    print (np.max(np.abs(d2alt)))
    if np.max(np.abs(d2alt)) > threshold_for_maneuver:
        is_maneuver = 1
    plane_speed = np.sqrt(np.power(dphi, 2) + np.power(dlambda, 2)) 
    
    plane_speed *= 111000
    
    dist = ((df['lat'].values[-1] - df['lat'].values[0]) ** 2 + (df['lon'].values[-1] - df['lon'].values[0]) ** 2)
    dist = np.sqrt(dist)
    
    f_list.append([filename, np.max(np.asarray(df['alt'].values, dtype = np.float64)), dalt_mean,
                   np.mean(plane_speed), dist, is_maneuver])

813.4739056032413
654.904534982571
10.000000000014552
0.0
15.022958443625612


In [3]:
f_list = np.asarray(f_list)

In [5]:
for idx, x in enumerate(['fname', 'alt_max', 'vert_acc', 'horiz_speed', 'dist', 'is_maneuver']):
    test_features[x] = f_list[:, idx]

In [6]:
test_features.describe()

,fname,alt_max,vert_acc,horiz_speed,dist,is_maneuver
count,5,5,5,5,5,5
unique,5,5,5,5,5,2
top,./test_files\traj21.csv,8281.973220666241,-9.999999999999998,876.349492666167,13.605878178530729,0
freq,1,1,1,1,1,3


In [8]:
test_features = test_features.sort_values(['fname'], ascending = [1], axis = 0)

In [9]:
rcs_mean = [80, 3, 0.01, 0.1, 15]

In [10]:
rcs_values = []
for i in rcs_mean:
    for k in np.random.normal(i, i/10, 1):
        rcs_values.append(k)

In [11]:
rcs_values

[84.55235581645529,
 3.201685279789068,
 0.011262761142884811,
 0.1089054904084633,
 14.669487356435447]

In [12]:
test_features['rcs'] = rcs_values

In [13]:
test_features.to_csv('test_features.csv', index = False)